## Paquetes y datos originales ## 

In [116]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')

### Previamente se ha tratado la base de datos y se han hecho 16 nuevas columnas para facilitar la lectura

In [117]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,6.0,6.0,None,NaN,NaN,NaN,NaN,NaN,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,7.0,7.0,4.0,49.900000,NaN,1.0,NaN,NaN,None,None


## Dividiremos los datos en 3 grupos:
- Con PA (codigo ICD de PA )
- Con PA + MECVV (MECVV positivo + icd pneumonia < 30 days)
- Sin PA (ICD pneumonia or icd pneumonia + mecvv positive > 30 days)

# CON PA 

In [118]:
tabla.PA_diagnosticada.value_counts()

PA_diagnosticada
0.0    33
1.0    17
Name: count, dtype: int64

In [131]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,BAIX
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,7.0,7.0,None,46.400000,NaN,1.0,1.0,1.0,NECTAR,MIG
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,6.0,6.0,None,NaN,7.0,1.0,1.0,1.0,PUDING,MIG
15,75,M,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-09-19 00...",[],...,13.0,13.0,None,NaN,NaN,NaN,NaN,NaN,None,None
26,97,F,0,"[{'date': '20220419', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'C09BA', 'dataInici': '2022-04-17 00...",[],...,5.0,5.0,None,NaN,NaN,NaN,NaN,NaN,None,None
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,6.0,6.0,None,0.000000,NaN,0.0,0.0,0.0,LIQUID,MIG
29,97,F,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2020-11-30 00...",[],...,14.0,14.0,None,NaN,NaN,NaN,NaN,NaN,None,None


# CON_PA + MECVV

In [132]:
CON_PA_MECVV = tabla[(tabla["Disfagia_mecvvs"] == 1.0) & (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
CON_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,BAIX
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,7.0,7.0,None,46.400000,NaN,1.0,1.0,1.0,NECTAR,MIG
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,6.0,6.0,None,NaN,7.0,1.0,1.0,1.0,PUDING,MIG
15,75,M,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-09-19 00...",[],...,13.0,13.0,None,NaN,NaN,NaN,NaN,NaN,None,None
26,97,F,0,"[{'date': '20220419', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'C09BA', 'dataInici': '2022-04-17 00...",[],...,5.0,5.0,None,NaN,NaN,NaN,NaN,NaN,None,None
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,6.0,6.0,None,0.000000,NaN,0.0,0.0,0.0,LIQUID,MIG
29,97,F,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2020-11-30 00...",[],...,14.0,14.0,None,NaN,NaN,NaN,NaN,NaN,None,None


# SIN_PA

In [133]:
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) | (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30)]
SIN_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,6.0,6.0,None,NaN,NaN,NaN,NaN,NaN,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,7.0,7.0,4.0,49.900000,NaN,1.0,NaN,NaN,None,None
5,57,M,0,"[{'date': '20220909', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],"[{'valor': '83.7', 'data': '2020-08-13'}, {'va...","[{'codi': 'N02BE', 'dataInici': '2020-08-12 00...",[],...,3.0,3.0,None,77.760000,NaN,NaN,NaN,NaN,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,BAIX
7,90,F,0,"[{'date': '20220627', 'name': 'GLUCOSA Sèrum',...","[{'imc': '0', 'anorexia': '0', 'perduaPes': '1...","[{'activitat': '3', 'estatMental': '1', 'humit...","[{'dataValoracio': '2020-07-30', 'horaValoraci...",[],"[{'codi': 'C09AA', 'dataInici': '2020-07-31 00...","[{'data': '20200731092102', 'disfagia': 'N', '...",...,12.0,12.0,1.0,NaN,9.0,0.0,0.0,0.0,LIQUID,MIG
8,90,F,0,"[{'date': '20220827', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BB', 'dataInici': '2023-07-17 00...",[],...,5.0,5.0,None,NaN,NaN,NaN,NaN,NaN,None,None
9,91,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C10AA', 'dataInici': '2020-08-25 00...",[],...,3.0,3.0,None,NaN,NaN,NaN,NaN,NaN,None,None
